<a href="https://colab.research.google.com/github/mhhmariana/Proyecto_IA_TalentoTech/blob/main/Pred_Guarde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar Librerias neceasarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Libreria para Google Drive
!pip install gdown
from google.colab import drive

In [ ]:
# https://drive.google.com/file/d/12cIekao280xCk665h2GZXMBdsbCucA9H/view?usp=sharing
# https://drive.google.com/file/d/1pnxqtEA0b3lf4nwg-6aj4KFxSAb-M0WP/view?usp=sharing
# https://drive.google.com/file/d/1X9sd_2WhoYopgsDy33_Xsvo2XpBMUXN7/view?usp=sharing
# https://drive.google.com/file/d/17Ng6GFfDaEghL46CO6ifHV5MB7_UKhcO/view?usp=sharing
# https://drive.google.com/file/d/1TwPNIe-LqaFZPflW23HQV_oHZjmVan7O/view?usp=sharing

# Lista de IDs de archivos de Google Drive para los diferentes años
# Reemplaza estos con los IDs reales de tus archivos CSV
file_ids = [
    '12cIekao280xCk665h2GZXMBdsbCucA9H', # Ejemplo de ID de archivo para un año
    '1pnxqtEA0b3lf4nwg-6aj4KFxSAb-M0WP', # Reemplaza con tu ID de archivo
    '1X9sd_2WhoYopgsDy33_Xsvo2XpBMUXN7', # Reemplaza con tu ID de archivo
    '17Ng6GFfDaEghL46CO6ifHV5MB7_UKhcO', # Reemplaza con tu ID de archivo
    '1TwPNIe-LqaFZPflW23HQV_oHZjmVan7O', # Reemplaza con tu ID de archivo
    # Agrega más IDs de archivo según sea necesario
]

# Lista para almacenar DataFrames individuales
dataframes_list = []

# Iterar a través de cada ID de archivo, descargar y leer el CSV
for file_id in file_ids:
    url = f'https://drive.google.com/uc?id={file_id}'
    try:
        temp_df = pd.read_csv(url)
        dataframes_list.append(temp_df)
        print(f"Datos leídos exitosamente del archivo con ID: {file_id}")
    except Exception as e:
        print(f"Error al leer los datos del archivo con ID: {file_id} - {e}")


# Concatenar todos los DataFrames en uno solo
if dataframes_list:
    data = pd.concat(dataframes_list, ignore_index=True)
    print("\n¡Todos los datos combinados exitosamente!")
else:
    data = pd.DataFrame()
    print("\nNo se cargaron datos.")

In [ ]:
#Exraer los datos del archivo y poderlos visualizar en una tabla ordenada
display(data)

In [ ]:
# Convertir la columna 'fecha' a datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# Extraer características de la fecha
data['año'] = data['fecha'].dt.year
data['mes'] = data['fecha'].dt.month
data['dia'] = data['fecha'].dt.day
data['dia_semana_num'] = data['fecha'].dt.dayofweek # Lunes=0, Domingo=6

# Codificar variables categóricas (dia_semana, motivo_estancia, condicion_climatica, evento_especial)
data_encoded = pd.get_dummies(data, columns=['dia_semana', 'motivo_estancia', 'condicion_climatica', 'evento_especial'], drop_first=True)

# Eliminar la columna 'fecha' original ya que hemos extraído las características
data_processed = data_encoded.drop('fecha', axis=1)

# Reordenar columnas para mostrar las de fecha primero
date_cols = ['año', 'mes', 'dia', 'dia_semana_num']
other_cols = [col for col in data_processed.columns if col not in date_cols]
data_processed = data_processed[date_cols + other_cols]

#Mostrar los datos procesados
display(data_processed)

In [ ]:
# Definir las variables objetivo y sus tipos (regresión o clasificación)
# Corregimos el nombre de la columna 'evento_especial' a 'evento_especial_True' después de la codificación one-hot
target_variables = {
    'numero_perros_teckel': 'regression',
    'estancias_largas': 'regression',
    'temperatura_celsius': 'regression',
    'dia_semana_Lunes': 'classification',
    'dia_semana_Martes': 'classification',
    'dia_semana_Miércoles': 'classification',
    'dia_semana_Jueves': 'classification',
    'dia_semana_Viernes': 'classification',
    'dia_semana_Sábado': 'classification',
    'condicion_climatica_Nublado': 'classification',
    'condicion_climatica_Soleado': 'classification',
    'motivo_estancia_Trabajo': 'classification',
    'motivo_estancia_Vacaciones': 'classification'
}

# Las características (X) serán todas las columnas en data_processed excepto las variables objetivo
features = [col for col in data_processed.columns if col not in target_variables.keys()]
X = data_processed[features]

# Diccionario para almacenar los modelos entrenados
trained_models = {}

# Importar los modelos necesarios
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Entrenar un modelo para cada variable objetivo
for target_variable, model_type in target_variables.items():
    y = data_processed[target_variable]

    if model_type == 'regression':
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_type == 'classification':
        # Para clasificación, necesitamos asegurarnos de que la variable objetivo es numérica (0 o 1)
        # y que tiene al menos 2 clases.
        if y.nunique() > 1:
            model = RandomForestClassifier(n_estimators=100, random_state=42)
        else:
            print(f"Saltando el entrenamiento para '{target_variable}' ya que solo tiene una clase.")
            continue
    else:
        print(f"Tipo de modelo desconocido para '{target_variable}'. Saltando.")
        continue

    model.fit(X, y)
    trained_models[target_variable] = model

print("Modelos entrenados con éxito para las variables objetivo especificadas.")

In [ ]:
from datetime import datetime
import numpy as np

# Función para preprocesar una fecha futura
def preprocess_future_date(future_date_str, X_columns):
    """
    Preprocesa una cadena de fecha futura para que coincida con el formato de las características de entrenamiento (X_columns).
    """
    future_date = pd.to_datetime(future_date_str)

    # Crear un DataFrame con la fecha futura y extraer características numéricas de la fecha
    future_data = pd.DataFrame({
        'año': [future_date.year],
        'mes': [future_date.month],
        'dia': [future_date.day],
        'dia_semana_num': [future_date.weekday()] # Lunes=0, Domingo=6
    })

    # Crear un DataFrame vacío con las mismas columnas que X
    future_processed = pd.DataFrame(columns=X_columns, index=[0])

    # Llenar las columnas numéricas de fecha
    for col in ['año', 'mes', 'dia', 'dia_semana_num']:
        if col in future_processed.columns:
            future_processed[col] = future_data[col]

    # Para las columnas one-hot encoded, establecer el valor a True para la categoría correspondiente
    # y False para las demás. Esto requiere inferir la categoría a partir de la fecha.

    # Inferir día de la semana
    day_of_week_map = {
        0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves',
        4: 'Viernes', 5: 'Sábado', 6: 'Domingo'
    }
    predicted_day_name = day_of_week_map[future_date.weekday()]
    if f'dia_semana_{predicted_day_name}' in future_processed.columns:
         future_processed[f'dia_semana_{predicted_day_name}'] = True
    # Si Domingo fue la columna eliminada por drop_first y es el día predicho,
    # todas las otras columnas dia_semana_ serán False, lo cual es correcto.

    # Para motivo_estancia, condicion_climatica, evento_especial,
    # no podemos inferirlos solo de la fecha. Necesitamos un enfoque diferente.
    # Una opción es asumir valores por defecto o predecirlos por separado si fuera posible
    # sin depender de otros factores que no sean la fecha.
    # Dado que el modelo de clasificación de Random Forest predice la probabilidad de cada clase,
    # podemos dejar que el modelo prediga estas categorías.

    # Por ahora, nos aseguramos de que las columnas one-hot encoded existan y sean False por defecto
    # (excepto la del día de la semana que acabamos de establecer si corresponde).
    for col in X_columns:
        if col not in future_processed.columns:
            future_processed[col] = False # Inicializar con False

    # Asegurar que todas las columnas sean numéricas (booleanos se tratan como 0/1)
    future_processed = future_processed.astype(float)


    return future_processed


# Función para realizar predicciones para una fecha futura
def predict_future(future_date_str, trained_models, X_columns):
    """
    Realiza predicciones para una fecha futura utilizando los modelos entrenados.
    Incluye una estimación de la desviación estándar para la predicción de perros.
    """
    future_data_processed = preprocess_future_date(future_date_str, X_columns)

    predictions = {}
    for target_variable, model in trained_models.items():
        if target_variable in ['numero_perros_teckel', 'estancias_largas', 'temperatura_celsius']:
            # Predicción para variables de regresión
            predictions[f'predicted_{target_variable}'] = model.predict(future_data_processed)[0]

            # Si es la predicción de perros, calcular la desviación estándar de las predicciones de los árboles
            if target_variable == 'numero_perros_teckel':
                 tree_predictions = [tree.predict(future_data_processed)[0] for tree in model.estimators_]
                 predictions['numero_perros_teckel_std_dev'] = np.std(tree_predictions)
                 # predictions['numero_perros_teckel_tree_predictions'] = tree_predictions # Opcional: guardar las predicciones de cada árbol

        elif model.__class__.__name__ == 'RandomForestClassifier':
            # Predicción para variables de clasificación
            # Predecir la probabilidad de cada clase
            probabilities = model.predict_proba(future_data_processed)[0]
            # Obtener el índice de la clase con mayor probabilidad
            predicted_class_index = np.argmax(probabilities)
            # Obtener el nombre de la clase predicha (True/False para columnas dummy)
            predicted_class_encoded = model.classes_[predicted_class_index]
            probability_predicted_class = probabilities[predicted_class_index]


            # Asignar las predicciones al diccionario
            if target_variable.startswith('dia_semana_'):
                 # Para el día de la semana, necesitamos encontrar cuál columna de día_semana_ tiene la mayor probabilidad de ser True
                 day_probs = {col.replace('dia_semana_', ''): model.predict_proba(future_data_processed)[0][1]
                              for col, model in trained_models.items() if col.startswith('dia_semana_') and model.__class__.__name__ == 'RandomForestClassifier'}
                 # Añadir Domingo si no fue dropeado y tiene su propio clasificador (aunque en este caso fue dropeado)
                 # Si Domingo fue dropeado, su "predicción" implícita es cuando todas las demás son False.
                 # Sin embargo, la forma más robusta es ver cuál de las columnas que *tenemos* tiene la mayor probabilidad de ser True.
                 # Si ninguna tiene alta probabilidad, podría sugerir la clase dropeada si es común.

                 # Una mejor manera: predecir la clase más probable entre TODAS las categorías originales
                 # Esto requiere entrenar un clasificador multiclase para 'dia_semana' en lugar de binarios para cada día.
                 # Dado que entrenamos clasificadores binarios, interpretaremos las probabilidades de ser True.

                 # Encontrar el día de la semana con la mayor probabilidad de ser True
                 predicted_day = 'Domingo' # Asumimos Domingo por defecto si todas las columnas dia_semana_ son False
                 max_day_prob = 0.0
                 for day_col in [col for col in trained_models.keys() if col.startswith('dia_semana_')]:
                     # Asegurarse de que el modelo para este día exista
                     if day_col in trained_models and trained_models[day_col].__class__.__name__ == 'RandomForestClassifier':
                          prob_true = trained_models[day_col].predict_proba(future_data_processed)[0][1]
                          if prob_true > max_day_prob:
                              max_day_prob = prob_true
                              predicted_day = day_col.replace('dia_semana_', '')

                 predictions['predicted_dia_semana'] = predicted_day
                 predictions['probability_dia_semana'] = max_day_prob # Probabilidad del día predicho

            elif target_variable.startswith('motivo_estancia_'):
                 motivo_probs = {col.replace('motivo_estancia_', ''): model.predict_proba(future_data_processed)[0][1]
                                 for col, model in trained_models.items() if col.startswith('motivo_estancia_') and model.__class__.__name__ == 'RandomForestClassifier'}
                 if motivo_probs:
                      predicted_motivo = max(motivo_probs, key=motivo_probs.get)
                      predictions['predicted_motivo_estancia'] = predicted_motivo
                      predictions['probability_motivo_estancia'] = motivo_probs[predicted_motivo]
                 else:
                      predictions['predicted_motivo_estancia'] = 'Desconocido'
                      predictions['probability_motivo_estancia'] = 0.0

            elif target_variable.startswith('condicion_climatica_'):
                 condicion_probs = {col.replace('condicion_climatica_', ''): model.predict_proba(future_data_processed)[0][1]
                                    for col, model in trained_models.items() if col.startswith('condicion_climatica_') and model.__class__.__name__ == 'RandomForestClassifier'}
                 if condicion_probs:
                      predicted_condicion = max(condicion_probs, key=condicion_probs.get)
                      predictions['predicted_condicion_climatica'] = predicted_condicion
                      predictions['probability_condicion_climatica'] = condicion_probs[predicted_condicion]
                 else:
                      predictions['predicted_condicion_climatica'] = 'Desconocido'
                      predictions['probability_condicion_climatica'] = 0.0

            elif target_variable == 'evento_especial_True':
                 # Para evento_especial_True, la predicción es si es True o False basada en la probabilidad
                 prob_true = model.predict_proba(future_data_processed)[0][1] # Probabilidad de que sea True
                 predictions['predicted_evento_especial'] = 'Sí' if prob_true > 0.5 else 'No' # Umbral simple del 50%
                 predictions['probability_evento_especial_True'] = prob_true # Probabilidad de que el evento especial sea True


    return predictions

# --- Ejemplo de uso (puedes cambiar la fecha aquí) ---
future_date_to_predict = str(input("Ingrese la fecha a pronosticar")) # '2025-12-20' Cambia esta fecha a la que desees predecir

# Pasamos X.columns a la función de preprocesamiento
future_date_prediction = predict_future(future_date_to_predict, trained_models, X.columns)

# Mostrar las predicciones
print(f"Predicciones para la fecha: {future_date_to_predict}")
print("-" * 30)
print(f"Día de la semana predicho: {future_date_prediction.get('predicted_dia_semana', 'N/A')} (Probabilidad: {future_date_prediction.get('probability_dia_semana', 0.0):.2f})")
print(f"Número de perros Teckel predicho: {future_date_prediction.get('predicted_numero_perros_teckel', 'N/A'):.2f}")
print(f"Desviación estándar de la predicción de perros: {future_date_prediction.get('numero_perros_teckel_std_dev', 'N/A'):.2f}")
print(f"Estancias largas predichas: {future_date_prediction.get('predicted_estancias_largas', 'N/A'):.2f}")
print(f"Temperatura predicha (Celsius): {future_date_prediction.get('predicted_temperatura_celsius', 'N/A'):.2f}")
print(f"Condición climática predicha: {future_date_prediction.get('predicted_condicion_climatica', 'N/A')} (Probabilidad: {future_date_prediction.get('probability_condicion_climatica', 0.0):.2f})")
print(f"Motivo de estancia predicho: {future_date_prediction.get('predicted_motivo_estancia', 'N/A')} (Probabilidad: {future_date_prediction.get('probability_motivo_estancia', 0.0):.2f})")
print(f"Evento especial predicho: {future_date_prediction.get('predicted_evento_especial', 'N/A')} (Probabilidad de True: {future_date_prediction.get('probability_evento_especial_True', 0.0):.2f})") # Cambiado a probability_evento_especial_True

In [ ]:
import csv
import random
from datetime import date, timedelta
from google.colab import drive # Importar drive

# Montar Google Drive
drive.mount('/content/drive')

# Definir el directorio en tu Google Drive donde quieres guardar el archivo
# Reemplaza 'Your_Folder_Name' con el nombre real de la carpeta en tu Drive
drive_path = '/content/drive/My Drive/Your_Folder_Name/simulated_data.csv'


# Nombres de las columnas
column_names = [
    "fecha",
    "dia_semana",
    "numero_perros_teckel",
    "estancias_largas",
    "motivo_estancia",
    "evento_especial",
    "temperatura_celsius",
    "condicion_climatica"
]

# Días de la semana en español
dias_semana = {
    0: "Lunes",
    1: "Martes",
    2: "Miércoles",
    3: "Jueves",
    4: "Viernes",
    5: "Sábado",
    6: "Domingo"
}

# Posibles motivos de estancia
motivos_estancia = ["Vacaciones", "Trabajo"]

# Posibles condiciones climáticas
condiciones_climaticas = ["Soleado", "Nublado", "Lluvia"]

# Función para generar datos simulados para un día
def generar_fila_datos(current_date):
    """
    Genera una fila de datos simulados para un día específico.
    Los valores se basan en la fecha y el día de la semana para mayor realismo.
    """
    day_of_week = current_date.weekday()
    dia_semana_str = dias_semana[day_of_week]

    # Mayor cantidad de perros en fines de semana y temporada alta
    is_weekend = day_of_week >= 5
    is_vacation_month = current_date.month in [6, 7, 12] # Junio, Julio, Diciembre

    if is_weekend or is_vacation_month:
        numero_perros = random.randint(10, 20)
        motivo = "Vacaciones"
    else:
        numero_perros = random.randint(5, 12)
        motivo = random.choice(motivos_estancia)

    # Estancias largas es un subconjunto de perros totales
    estancias_largas = random.randint(3, numero_perros - 2) if numero_perros > 5 else 0

    # Evento especial (aleatorio, puede coincidir con festivos)
    evento_especial = "Sí" if random.random() < 0.05 else "No"

    # Simular un aumento de temperatura en meses cálidos
    if current_date.month in [6, 7, 8]:
        temperatura = random.randint(28, 35)
    elif current_date.month in [1, 2, 11, 12]:
        temperatura = random.randint(18, 25)
    else:
        temperatura = random.randint(22, 30)

    # Condición climática aleatoria
    condicion_climatica = random.choice(condiciones_climaticas)

    # Devolver la fila como una lista de valores
    return [
        current_date.strftime("%Y-%m-%d"),
        dia_semana_str,
        numero_perros,
        estancias_largas,
        motivo,
        evento_especial,
        temperatura,
        condicion_climatica
    ]

# Configuración del año 2020 (que es un año bisiesto)
start_date = date(2020, 1, 1)
end_date = date(2020, 12, 31)
delta = timedelta(days=1)

# Crear el contenido CSV
csv_data = []
csv_data.append(column_names)

current_date = start_date
while current_date <= end_date:
    csv_data.append(generar_fila_datos(current_date))
    current_date += delta

# Guardar los datos en un archivo CSV en Google Drive
with open(drive_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(csv_data)

print(f"Datos guardados en {drive_path}")

### 1. Exploración Inicial de Datos
Vamos a mostrar las estadísticas descriptivas y la estructura del DataFrame `data_processed` para tener una primera idea de los datos.

In [ ]:
# Mostrar las primeras filas del DataFrame
print("Primeras 5 filas del DataFrame data_processed:")
display(data_processed.head())

# Mostrar información general del DataFrame (tipos de datos, valores no nulos)
print("\nInformación del DataFrame data_processed:")
data_processed.info()

# Mostrar estadísticas descriptivas para las columnas numéricas
print("\nEstadísticas descriptivas del DataFrame data_processed:")
display(data_processed.describe())

### 2. Análisis de Correlación
Vamos a calcular la matriz de correlación para las variables numéricas y visualizarla usando un mapa de calor. Esto nos ayudará a identificar relaciones lineales entre las variables.

In [ ]:
# Seleccionar solo las columnas numéricas para el análisis de correlación
# Excluimos las columnas booleanas que representan variables categóricas codificadas
numeric_cols = data_processed.select_dtypes(include=np.number)

# Calcular la matriz de correlación
correlation_matrix = numeric_cols.corr()

# Mostrar la matriz de correlación
print("Matriz de correlación de las variables numéricas:")
display(correlation_matrix)

# Visualizar la matriz de correlación usando un mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación de Variables Numéricas')
plt.show()

### 3. Análisis de Variables Categóricas

Vamos a examinar la distribución de las variables categóricas codificadas y su relación con algunas de las variables objetivo clave como `numero_perros_teckel` y `estancias_largas`. Utilizaremos gráficos de barras y/o box plots para visualizar estas relaciones.

In [ ]:
# Variables categóricas codificadas de interés (excluyendo las que ya son variables objetivo directas)
categorical_features = [col for col in data_processed.columns if data_processed[col].dtype == 'bool' and col not in target_variables.keys()]

print("Análisis de variables categóricas y su relación con las variables objetivo:")

# Analizar la relación de cada variable categórica con las variables objetivo clave
target_reg_vars = ['numero_perros_teckel', 'estancias_largas']

for cat_col in categorical_features:
    print(f"\nAnálisis para la variable categórica: {cat_col}")

    for target_var in target_reg_vars:
        plt.figure(figsize=(8, 5))
        # Usamos boxplot para ver la distribución de la variable objetivo para cada categoría (True/False)
        sns.boxplot(x=cat_col, y=target_var, data=data_processed)
        plt.title(f'{target_var} vs {cat_col}')
        plt.xlabel(cat_col)
        plt.ylabel(target_var)
        plt.show()

# Además, podemos ver la distribución de las categorías originales antes de la codificación one-hot
# Esto requiere usar el DataFrame 'data' original
print("\nDistribución de las variables categóricas originales:")
for original_cat_col in ['dia_semana', 'motivo_estancia', 'condicion_climatica', 'evento_especial']:
    if original_cat_col in data.columns:
        print(f"\nDistribución de '{original_cat_col}':")
        display(data[original_cat_col].value_counts())
        plt.figure(figsize=(8, 5))
        sns.countplot(x=original_cat_col, data=data, order=data[original_cat_col].value_counts().index)
        plt.title(f'Distribución de {original_cat_col}')
        plt.xlabel(original_cat_col)
        plt.ylabel('Frecuencia')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

### Visualización de la Cantidad de Perros Teckel a lo Largo del Tiempo

Vamos a crear un gráfico de series de tiempo para visualizar la cantidad diaria de perros Teckel (`numero_perros_teckel`) a lo largo de todas las fechas disponibles en el conjunto de datos. Esto nos ayudará a identificar tendencias, estacionalidad o picos inusuales en la cantidad de perros.

In [ ]:
# Asegurarnos de que la columna 'fecha' sea de tipo datetime (ya lo hicimos, pero es buena práctica confirmarlo)
data['fecha'] = pd.to_datetime(data['fecha'])

# Establecer la columna 'fecha' como índice para facilitar el trazado de series de tiempo
data_sorted = data.sort_values('fecha')

# Crear el gráfico de series de tiempo
plt.figure(figsize=(15, 7))
sns.lineplot(x='fecha', y='numero_perros_teckel', data=data_sorted)
plt.title('Cantidad de Perros Teckel a lo Largo del Tiempo')
plt.xlabel('Fecha')
plt.ylabel('Número de Perros Teckel')
plt.grid(True)
plt.show()